<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/SimpleMathSolverVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program is an interactive math-solving tool designed to assist users in solving algebraic equations step-by-step with the help of OpenAI's GPT. Users begin by entering a math question and a verification method, such as substituting the solution back into the equation. The program uses GPT to generate a detailed explanation of the solution and then verifies the correctness of the final answer by calling GPT again. The output includes reasoning steps, the final answer, and the verification result. To use the program, ensure you have an OpenAI API key and a properly configured environment. Possible extensions include adding support for other types of math problems (e.g., calculus or geometry), enabling graphical representations of solutions, and providing feedback on user-defined verifying methods.

Here I enter the following problem and verification methd:
Enter a math question (e.g., how can I solve 8x + 7 = -23): using Simpson’s Rule to calculate integral of x^2 from 1 to 2
Enter a verifying method (e.g., substitute your final answer into both sides of the equation): compared with analytical solution,

In [14]:
from pydantic import BaseModel
from openai import OpenAI

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

# Initialize OpenAI client
client = OpenAI(api_key="sk-proj-****A")

def solve_math_question():
    # Step 1: Ask the user to input a math question
    math_question = input("Enter a math question (e.g., how can I solve 8x + 7 = -23): ")

    # Step 2: Ask the user to input a verifying method
    verifying_method = input("Enter a verifying method (e.g., substitute your final answer into both sides of the equation): ")

    # Step 3: Solve the math question using OpenAI
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
            {"role": "user", "content": math_question},
        ],
        response_format=MathReasoning,
    )

    # Parse the response
    math_reasoning = completion.choices[0].message.parsed

    # Step 4: Output the math reasoning steps
    print("\nMath Reasoning Steps:")
    for step in math_reasoning.steps:
        print(f"Explanation: {step.explanation}")
        print(f"Output: {step.output}\n")

    # Step 5: Output the final answer
    final_answer = math_reasoning.final_answer
    print(f"Final Answer: {final_answer}\n")

    # Step 6: Verify the result using GPT
    print("Now let's verify the solution.")
    verification_prompt = (
        f"Verify the solution to the question '{math_question}'. "
        f"The final answer provided was {final_answer}. {verifying_method}. "
        "Confirm whether the solution is correct."
    )

    verification_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a math verification assistant. Confirm if the solution is correct."},
            {"role": "user", "content": verification_prompt},
        ]
    )

    verification_response = verification_completion.choices[0].message.content
    print("Verification Result:")
    print(verification_response)

# Run the program interactively
if __name__ == "__main__":
    solve_math_question()


Enter a math question (e.g., how can I solve 8x + 7 = -23): using Simpson’s Rule to calculate integral of x^2 from 1 to 2
Enter a verifying method (e.g., substitute your final answer into both sides of the equation): compared with analytical solution,

Math Reasoning Steps:
Explanation: Identify the function and the limits for the integration. Here, we have f(x) = x^2, a = 1, b = 2, and we need to calculate the integral from a to b.
Output: f(x) = x^2, a = 1, b = 2.

Explanation: Simpson's Rule states that the integral can be approximated as: \[\int_a^b f(x) dx \approx \frac{h}{3} \left( f(a) + 4f\left(\frac{a+b}{2}\right) + f(b) \right]\] where h = (b - a)/2, which is the width of the subintervals.
Output: h = (2 - 1) / 2 = 0.5.

Explanation: Now we need to calculate f(a), f(b), and f((a+b)/2): \[f(1) = 1^2 = 1, \quad f(2) = 2^2 = 4, \quad f\left(\frac{1+2}{2}\right) = f(1.5) = (1.5)^2 = 2.25.\]
Output: f(1) = 1, f(2) = 4, f(1.5) = 2.25.

Explanation: Now substitute these values back 